## Filter Action-Object Pairs

In [1]:
import pandas as pd
from collections import Counter
import spacy
from spacy.matcher import Matcher
from tqdm import tqdm
from IPython.display import display, HTML
tqdm.pandas()


# Load the English model
nlp = spacy.load("en_core_web_lg")

In [2]:
df = pd.read_parquet('../../data/processed/targets/avocado_train_targets_exploded.parquet', engine='fastparquet')
df = df.rename(columns={'targets': 'target'})


In [3]:
df.shape

(1205714, 19)

In [4]:
# Remove empty entries

df_filtered = df[~df['action_object_pairs'].apply(lambda x: x is None or (isinstance(x, list) and len(x) == 0))]

In [5]:
df_filtered.shape

(722472, 19)

### Inspect Action-Object-Pairs results

In [6]:
def print_distinct_words(df,n=20):
    all_words = [word for sublist in df['action_object_pairs'] for word in sublist]

    # Use Counter to count occurrences of each word
    word_counts = Counter(all_words)

    # Convert to DataFrame (optional, if you want to keep it in tabular form)
    distinct_words_df = pd.DataFrame(word_counts.items(), columns=['Word', 'Count'])
    distinct_words_df = distinct_words_df.sort_values(by='Count', ascending=False)
    display(HTML(distinct_words_df.head(n).to_html()))

In [7]:
print_distinct_words(df_filtered, n=50)

,Word,Count
20,start_Server,12831
63,send_it,12052
177,have_questions,11024
64,send_message,10704
301,send_email,8186
19,fail_Message,8065
18,start_Failures,6983
376,start_Occurrences,6428
395,call_me,6177
103,thank_you,4472


### Inspect Messages


| Action-Object-Pair    | Count |
|-------------|-------------|
 start_Server |	12831
send_it |	12052
have_questions |	11024
send_message |	10704
send_email |	8186
fail_Message |	8065
start_Failures |	6983
start_Occurrences |	6428
call_me	| 6177
thank_you | 4472
contact_me | 3995
post_message | 3790
miss_UNIVERSE | 3742
do_what | 3584
send_mail | 3490
start_occurrence | 3152
do_it | 2816
give_call | 2642
need_help | 2541
unsubscribe_mailto | 2468

In [8]:
def inspect_action_object_pair(pair_to_inspect, entry, print_extracted_text=True, print_target=True):
    test = df_filtered[df_filtered['action_object_pairs'].apply(lambda x: pair_to_inspect in x)]
    if print_extracted_text:
        print(test.iloc[entry]['extracted_text'])
    if print_target:
        if print_extracted_text:
            print('----------------------')
    print(test.iloc[entry]['target'])
    del test

In [9]:
inspect_action_object_pair('send_e', 1,print_extracted_text=False)

I might have sent the e-mail to the wrong person because you mentioned before that we should have a meeting with Mark


## Entry Removal

After inspecting the most-frequently occurring action-object-pairs, the following ones will not be considered further:

| Action-Object-Pair    | Reason |
|-------------|-------------|
|fail_Message|belongs to error message and thus does not contain a humanly intent|
|start_Failures|belongs to error message and thus does not contain a humanly intent|
|start_Occurrences|belongs to error message and thus does not contain a humanly intent|
|post_message|belongs to an automated message|
|unsubscribe_mailto|belongs to an automated message|
|miss_UNIVERSE|belongs to an automated message|
|start_occurrence|belongs to an automated message|

also the extremeprogramming unsubscribe message is being filtered from the original dataframe, as well as java messages

In [10]:
entries_to_remove = [
    'fail_Message',
    'start_Failures',
    'start_Occurrences',
    'post_message',
    'unsubscribe_mailto',
    'miss_UNIVERSE',
    'start_occurrence'
]

# Remove these entries
df_filtered = df_filtered[~df_filtered['action_object_pairs'].apply(lambda x: any(item in entries_to_remove for item in x))]

In [11]:
df_filtered.shape

(699454, 19)

In [12]:
print_distinct_words(df_filtered)

,Word,Count
18,start_Server,12831
173,have_questions,11024
297,send_email,8182
61,send_it,6190
390,call_me,6177
62,send_message,4856
100,thank_you,4433
230,contact_me,3994
405,do_what,3567
669,send_mail,3490


### Group Pairs